# Install openai-agents SDK

In [ ]:
!pip install -Uq openai-agents

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.5/128.5 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 567.4/567.4 kB 12.5 MB/s eta 0:00:00


# Make your Jupyter Notebook capable of running asynchronous functions.

In [ ]:
import nest_asyncio
nest_asyncio.apply()

# Run Google Gemini with OPENAI-Agent SDK

In [ ]:
import os
from agents import Agent, Runner, AsyncOpenAI, OpenAIChatCompletionsModel
from agents.run import RunConfig
from google.colab import userdata


gemini_api_key = userdata.get("GEMINI_API_KEY")


# Check if the API key is present; if not, raise an error
if not gemini_api_key:
    raise ValueError("GEMINI_API_KEY is not set. Please ensure it is defined in your .env file.")

#Reference: https://ai.google.dev/gemini-api/docs/openai
external_client = AsyncOpenAI(
    api_key=gemini_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)

model = OpenAIChatCompletionsModel(
    model="gemini-2.0-flash",
    openai_client=external_client
)

config = RunConfig(
    model=model,
    model_provider=external_client,
    tracing_disabled=True
)

In [ ]:
import os
from agents import Agent, Runner, AsyncOpenAI, OpenAIChatCompletionsModel, set_default_openai_client
from agents.run import RunConfig
from google.colab import userdata


gemini_api_key = userdata.get("GEMINI_API_KEY")


external_client = AsyncOpenAI(
    api_key=gemini_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)
set_default_openai_client(external_client)

# Tools
The OpenAI Agents SDK provides a robust framework for integrating various tools into agents, enabling them to perform tasks such as data retrieval, web searches, and code execution. Here's an overview of the key points regarding tool integration:

**Types of Tools:**

1. **Hosted Tools:** These are pre-built tools running on OpenAI's servers, accessible via the [`OpenAIResponsesModel`]. Examples include:
   - **WebSearchTool:** Enables agents to perform web searches.
   - **FileSearchTool:** Allows retrieval of information from OpenAI Vector Stores.
   - **ComputerTool:** Facilitates automation of computer-based tasks.

2. **Function Calling:** This feature allows agents to utilize any Python function as a tool, enhancing their versatility.

3. **Agents as Tools:** Agents can employ other agents as tools, enabling hierarchical task management without transferring control.

**Implementing Tools:**

- **Function Tools:** By decorating Python functions with `@function_tool`, they can be seamlessly integrated as tools for agents.

**Tool Execution Flow:**

- During an agent's operation, if a tool call is identified in the response, the SDK processes the tool call, appends the tool's response to the message history, and continues the loop until a final output is produced.

**Error Handling:**

- The SDK offers mechanisms to handle errors gracefully, allowing agents to recover from tool-related issues and continue their tasks effectively.

For a comprehensive understanding and implementation details, refer to the [tools documentation](https://github.com/openai/openai-agents-python/blob/main/docs/tools.md).

# Create tools

In [ ]:
from agents.tool import function_tool

In [ ]:
@function_tool("get_weather")
def get_weather(location: str, unit: str = "C") -> str:
  """
  Fetch the weather for a given location, returning a short description.
  """
  # Example logic
  return f"The weather in {location} is 22 degrees {unit}."

In [ ]:
@function_tool("piaic_student_finder")
def student_finder(student_roll: int) -> str:
  """
  find the PIAIC student based on the roll number
  """
  data = {1: "Qasim",
          2: "Sir Zia",
          3: "Daniyal"}

  return data.get(student_roll, "Not Found")

# Connect tools with Agent

In [ ]:
import asyncio

from agents import Agent, Runner


async def main():
    agent = Agent(
        name="Assistant",
        instructions="You only respond in haikus.",
        tools=[get_weather, student_finder], # add tools here
        model=model
    )

    result = await Runner.run(agent, "Share PIAIC roll no1 student details.")
    print(result.final_output)



if __name__ == "__main__":
    asyncio.run(main())

Qasim is the name.
Roll number one is his own.
A bright future dawns.

